In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess images using ImageDataGenerator
image_size = (32, 32)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    '/path/to/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Adjust for your task
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    '/path/to/test',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Adjust for your task
    shuffle=False
)

# Extract features for K-means clustering
num_centers = 50
kmeans_data = []

for _ in range(len(train_generator)):
    inputs, _ = train_generator.next()
    kmeans_data.append(inputs.reshape(inputs.shape[0], -1))

kmeans_data = np.concatenate(kmeans_data, axis=0)

# K-means clustering to find centers
kmeans = KMeans(n_clusters=num_centers, random_state=42)
kmeans.fit(kmeans_data)
initial_centers = kmeans.cluster_centers_

# Radial Basis Function (RBF) layer
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, num_centers):
        super(RBFLayer, self).__init__()
        self.num_centers = num_centers

    def build(self, input_shape):
        self.centers = self.add_weight("centers", (self.num_centers, np.prod(input_shape[1:])),
                                       initializer=tf.constant_initializer(initial_centers),
                                       trainable=False)

    def call(self, inputs):
        diff = tf.expand_dims(inputs, axis=1) - self.centers
        distances = tf.norm(diff, axis=-1)
        rbf = tf.exp(-distances**2)
        return rbf

# Build the RBF Network
num_classes = len(train_generator.class_indices)
model = models.Sequential([
    RBFLayer(num_centers),
    layers.Dense(num_classes, activation='softmax')  # Adjust for your task
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5, validation_data=test_generator)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
